In [30]:
# importing values
import numpy as np
from scipy.optimize import minimize
import pandas as pd

import csv

# code for getting user input from console
inputParameters =   input("Please enter character and enemy data in the form (CharName,CharLVL,SM,EnemyName,EnemyLVL,WeaponName,WeaponLVL)")

split_string = inputParameters.split(",")

#extracting values from string list
charName = split_string[0]
charLVL = float(split_string[1])
skillMultiplier = float(split_string[2])
enemyName =  split_string[3]
enemyLVL = float(split_string[4])
weaponName = split_string[5]
weaponLVL = float(split_string[6])

#making intermediate variable global to the script
CharBaseATK = 0
CharATKB = 0
CharCD = 0
CharCR = 0
CharEB = 0

WeapBaseATK = 0
WeapATKB = 0
WeapCD = 0
WeapCR = 0
WeapEB = 0

# opening CSV with character data and putting it into a dataframe
with open('CharLVL90.csv', mode='r') as csv_file:
    reader = csv.reader(csv_file)
    data = list(reader)
    print(data.index)
    dfC = pd.DataFrame(data, columns = ['Name','Base ATK','ATK%', 'CR','CD','EB','Type'])
    #print (df)  #for printing the CSV data if you want to see it

# opening CSV with weapon data and putting it into a dataframe
with open('WeaponLVL90Phys.csv', mode='r') as csv_file:
    reader = csv.reader(csv_file)
    data = list(reader)
    print(data.index)
    dfW = pd.DataFrame(data, columns = ['Name','Base ATK','ATK%', 'CR','CD','EB'])

#print(dfC)
#print(dfW)

# populating character variables with csv data
for i in range(31) :
    if str(dfC.iloc[i+1, 0]) == str(charName) :
        CharBaseATK = float(dfC.iloc[i+1, 1]) # character Base Atk
        CharATKB = float(dfC.iloc[i+1, 2])
        CharCR = float(dfC.iloc[i+1, 3])
        CharCD = float(dfC.iloc[i+1, 4])
        CharEB = float(dfC.iloc[i+1, 5]) 
        print('Character Stats:', 'Base ATK = ', CharBaseATK,'CritRate = ',  CharCR,'CritDMG = ',  CharCD,'ElemBonus = ',  CharEB)

# populating weapon variables with csv data
for i in range(55) :
    if str(dfW.iloc[i+1, 0]) == str(weaponName) :
        WeapBaseATK = float(dfW.iloc[i+1, 1]) # character Base Atk
        WeapATKB = float(dfW.iloc[i+1, 2])
        WeapCR = float(dfW.iloc[i+1, 3])
        WeapCD = float(dfW.iloc[i+1, 4]) 
        WeapEB = float(dfW.iloc[i+1, 5]) 
        print('Weapon Stats:','Base ATK = ', WeapBaseATK,'CritRate = ', WeapCR,'CritDMG = ', WeapCD,'ElemBonus = ', WeapEB)

TotalBaseATK = CharBaseATK + WeapBaseATK # finding total base attack

# filling other variables with either hardcode or user input
SM = skillMultiplier # skill multiplier (assuming LVL 6 charged atk)
CharLVL = charLVL
EnemyLVL = enemyLVL
DefDrop = 0
EnemyRes = 0.1 # Hilichurl Elemental Resistance
BaseHP = 13103 # Keqing base HP
EB = 0.466+CharEB+WeapEB+0.77

C1 = (1+EB)*(SM)*((100+CharLVL)/(100 + CharLVL + 100 + EnemyLVL)) *(1-EnemyRes) #calculating C1
C2 = TotalBaseATK # setting C2 to the total base attack

print("C1 = " + str(C1))
print("C2 = " + str(C2))

def objective(x) :

    a = x[0] #ATK%
    f = x[1] #flat atk
    z = x[2] #HP%
    h = x[3] #flat hp
    r = x[4] #crit rate
    d = x[5] #crit dmg


    HPatkBonusES = (0.0506)*(BaseHP*(0.466+1+0.0495*z)+254*h+4780)

    # conidtional for reaching max HP bonus cap
    if (HPatkBonusES > 4*C2) :
        HPatkBonusES = 4*C2
    
    #print("HP ES bonus max = " + str(4*C2))
    #print("HP ES bonus = " + str(HPatkBonusES))

    #print((-1)*C1*(C2*(1+0.466+CharATKB+WeapATKB+0.0495*a)+311+f*16.5+HPatkBonusES))
    #print(1+(0.311+WeapCR+CharCR+0.033*r)*(WeapCD+CharCD+0.066*d))

#assuming CR% / EB% / ATK%
    return (-1)*C1*(C2*(1+CharATKB+WeapATKB+0.0495*a)+311+f*16.5)*(1+(0.311+WeapCR+CharCR+0.033*r)*(WeapCD+CharCD+0.066*d))

    #return (-1)*(0.5)*C1*(C2*(1+0.466+0+0+0.0495*a)+311+f*16.5+HPatkBonusES)*(1+(0.311+0+0.05+0.033*r)*(0.662+0.884+0.066*d))

def constraint1(x) :
    return x[0]+x[1]+x[2]+x[3]+x[4]+x[5]-20

def printStats(x) :

    a = x[0] #ATK%
    f = x[1] #flat atk
    z = x[2] #HP%
    h = x[3] #flat hp
    r = x[4] #crit rate
    d = x[5] #crit dmg

    ATK = C2 + C2*(1+0.466+CharATKB + WeapATKB + 0.0495*a)+311+f*16.5
    #+0.018*(BaseHP*(1+0.0495*z)+254*h+4780)
    HP = BaseHP*(1+0.0495*z)+254*h+4780
    CR = 0.331+CharCR+WeapCR+0.033*r
    CD = CharCD+WeapCD+0.066*d

    #print("Avg Optimized DMG = " + str(objective(sol.x)))
    #print("Total ATK = " + str(ATK))
    #print("Total HP = " + str(HP))
    #print("Total CR = " + str(CR))
    #print("Total CD = " + str(CD))

    print(str((-1)*objective(sol.x)))
    print(str(ATK))
    #print(str(HP))
    print(str(CR))
    print(str(CD))

#x0 = [1,1,1,1,1,1]
x0 = [0,0,0,18,0,0] # initialization
print(objective(x0))

lowBound = (0.0, 20.0)
highBound = (0.0, 2)
bnds = (lowBound,lowBound,lowBound,lowBound,lowBound,lowBound)
con1 = {'type': 'eq', 'fun': constraint1}

cons = [con1]

sol = minimize(objective, x0, method = 'SLSQP', bounds=bnds, constraints = cons)

print(sol)
printStats(sol.x)


<built-in method index of list object at 0x000001BB7FAB4840>
<built-in method index of list object at 0x000001BB7FBC9E80>
Character Stats: Base ATK =  323.0 CritRate =  0.05 CritDMG =  0.884 ElemBonus =  0.0
Weapon Stats: Base ATK =  674.0 CritRate =  0.0 CritDMG =  0.0 ElemBonus =  0.413
C1 = 1.4900625
C2 = 997.0
-2962.9110791938497
     fun: -4918.821673108249
     jac: array([-105.47296143,  -35.26348877,    0.        ,    0.        ,
       -105.47314453, -105.47351074])
 message: 'Optimization terminated successfully'
    nfev: 67
     nit: 8
    njev: 8
  status: 0
 success: True
       x: array([16.09169456,  0.        ,  0.        ,  0.        ,  3.18144983,
        0.72685565])
4918.821673108249
3763.1512638772056
0.48598784449548715
0.9319724729719848
